# FULL PIPELINE


This notebook will merge four steps:

1 -  segment electrodes from fluoroscopy return coordinates

2 - segment electodes from CT return coordinates

3 -  map CT to 2D

4 -  map coordinates back to 3D

## STEP 1: SEGMENT FROM FLUORO

In [ ]:
# Run first component script
from fluoro_segmentation import fluoro_get_coordinates
coords_dict = fluoro_get_coordinates(fluoro)

In [ ]:
coords_dict['ecog'] = np.array(coords_dict['ecog'])

### Visual Check

In [ ]:
# Display segmented features on fluoro image
import matplotlib.pyplot as plt
plt.imshow(fluoro,cmap='gray')
plt.scatter(coords_dict['ecog'][:,0], coords_dict['ecog'][:,1])
plt.scatter(coords_dict['dbs'][0], coords_dict['dbs'][1])
plt.scatter(coords_dict['pin'][:,0], coords_dict['pin'][:,1])

## STEP 2: SEGMENT FROM CT

In [ ]:
# Run second component script
from ct_segmentation import ct_get_landmarks
coords3d_dic = ct_get_landmarks(prect_data, postct_data)
pins_ct = coords3d_dic['pin']

### Visual Check

In [ ]:
# Display the segmented features on CT volume
from mayavi import mlab
mlab.init_notebook()
fig = mlab.figure(bgcolor=(1, 1, 1), size=(500, 500))
source_1 = mlab.pipeline.scalar_field(prect_data)
surface = mlab.pipeline.iso_surface(source_1, 
                          contours=[256,], 
                          opacity=0.5, 
                          colormap = 'black-white')

source_2 = mlab.pipeline.scalar_field(postct_data[:,:128,:])
surface = mlab.pipeline.iso_surface(source_2, 
                          contours=[256,], 
                          opacity=0.8, 
                          colormap = 'black-white')
mlab.points3d(coords3d_dic['pin'][:,0],coords3d_dic['pin'][:,1],coords3d_dic['pin'][:,2], color = (0.2,1,.2), scale_factor=10)
mlab.points3d(coords3d_dic['lead'][:,0],coords3d_dic['lead'][:,1],coords3d_dic['lead'][:,2], color = (1,0,0), scale_factor=10)

## STEP 3: MAP TO 2D

When converting from pixel to 3d coordinate mayavi just centers x and y, so (1400,1200) image will have a center at (700,600) and possibly x and y switched

In [ ]:
# Refine inputs to 3rd component function

dbs = np.array([coords_dict['dbs']])
pins2d = coords_dict['pin']
pins_fl = np.concatenate((dbs,pins2d),axis=0)
coords_2d = coords_dict['ecog']
pins_ct = coords3d_dic['pin']
print(coords_2d)

In [ ]:
# Run third component function
from fluoro_ct_alignment import project_to_2d
aligned_coords = project_to_2d(postct_data,fluoro,pins_fl,pins_ct,coords_2d)

### Visual Inspect

In [ ]:
# Display aligned electrode coordinates on blank image space
plt.imshow(np.zeros([256,256]),cmap='gray')
plt.scatter(aligned_coords[:,0],aligned_coords[:,1],color='orange')
print(aligned_coords)

## STEP 4: MAP TO 3D

In [ ]:
# Run fourth component function
from hull_projection import project_to_3d
predictions = project_to_3d(aligned_coords, points_hull, Tmatrix)
predictions = np.array(predictions)
predictions

### Visual Inspect

In [ ]:
# Inspect the projected coordinates on the MRI hull
import matplotlib.pyplot as plt

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(points_hull[:,0], points_hull[:,1], points_hull[:,2], cmap='green', alpha=0.2)
ax.scatter3D(predictions[:,0], predictions[:,1], predictions[:,2], c='r',)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z');

For a better visualization, use plotly or mayavi. Here for the sake of fast visualization we use matplotlib.